In [1]:
import pandas as pd
import numpy as np
import scipy.special

In [ ]:
# train = {'X1': pd.Series([0, 0, 1, 1]),
#          'X2': pd.Series([0, 1, 0, 1])}
# X = pd.DataFrame(train)
# target =  pd.Series([0, 1, 1, 1])

In [ ]:
# X

,X1,X2
0,0,0
1,0,1
2,1,0
3,1,1


In [4]:
class neuralNetwork:

    def __init__(self, input_nodes, hidden_nodes, output_nodes, lr, x0):
        self.inodes = input_nodes # кол-во входных узлов
        self.hnodes = hidden_nodes # кол-во скрытых узлов
        self.onodes = output_nodes # кол-во выходных узлов
        self.x0 = x0 # смещение

        self.lr = lr

        self.wih = np.array([[1, 0.5],
                             [-1, 2]])
        self.who = np.array([1.5, -1])
        self.wbias_hidden = np.array([[1.0],
                                     [1.0]])
        self.wbias_output = np.array([1.0])

        # np.random.seed(42)
        # self.wih = np.random.random((self.hnodes, self.inodes)) # веса входного слоя
        # self.who = np.random.random((self.onodes, self.hnodes)) # веса скрытого слоя
        # self.wbias_hidden = np.random.random(self.hnodes) # вес смещения для узлов среднего слоя
        # self.wbias_output = np.random.random(self.onodes) # вес смещения для узла выходного слоя

        self.activation_function = lambda x: scipy.special.expit(x) # функция активации (сигмоида)

    def moving_forward(self, input_list): # расчет входных сигналов по сети
         # превращаем список в матрицу и транспонируем ее для правильного умножения значений нейронов на веса
        inputs = np.array(input_list, ndmin=2).T

        hidden_inputs = np.dot(self.wih, inputs) + self.x0 * self.wbias_hidden # вычисляем нейроны скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs) # применяем к скрытому слою функцию активации

    
        final_inputs = np.dot(self.who, hidden_outputs) + self.x0 * self.wbias_output # вычисляем нейроны выходного слоя
        final_output = self.activation_function(final_inputs) # применяем функцию активации
        
        return hidden_outputs, final_output

    def calc_error(self, input_list, target):
        hidden_outputs, final_output = self.moving_forward(input_list)

        error = pow((target - final_output), 2) / 2 # вычисляем ошибку
        return error

    # обучение модели с помощью обратного распространения ошибки
    def train(self, input_list, target): 
        hidden_outputs, final_output = self.moving_forward(input_list) # ищем выходное значение
        
        error = target - final_output # считаем производную квадратичной ошибки
        final_error = final_output * (1 - final_output) * error # считаем ошибку выходного узла используя производнную сигмоиды и квадратичной ошибки

        # корректируем веса и смещения из скрытого слоя в выходной
        self.who[0] += (self.lr * final_error * hidden_outputs[0])
        self.who[1] += (self.lr * final_error * hidden_outputs[1])
        self.wbias_output[0] += (self.lr * final_error * self.x0)

        hidden_error_1 = final_output * (1 - final_output) * self.who[0] * final_error # считаем ошибку первого узла скрытого слоя
        hidden_error_2 = final_output * (1 - final_output) * self.who[1] * final_error # считаем ошибку второго узла скрытого слоя

        # корректируем веса и смещения из входного слоя в скрытый
        self.wih[0][0] += (self.lr * hidden_error_1 * input_list[0])
        self.wih[0][1] += (self.lr * hidden_error_1 * input_list[1])
        self.wih[1][0] += (self.lr * hidden_error_2 * input_list[0])
        self.wih[1][1] += (self.lr * hidden_error_2 * input_list[1])
        self.wbias_hidden[0] += (self.lr * hidden_error_1 * self.x0)
        self.wbias_hidden[1] += (self.lr * hidden_error_2 * self.x0)

In [5]:
nn = neuralNetwork(2, 2, 1, 0.5, 1)

In [6]:
nn.calc_error([0,1], 1)

array([0.02389507])

In [7]:
nn.train([0,1], 1)

In [8]:
nn.calc_error([0, 1], 1)

array([0.02207046])

In [10]:
nn.moving_forward([0, 1])

(array([[0.81901166],
        [0.95229027]]),
 array([0.78990258]))